# Lagged Rain

Let's plot the time evolution of the atmosphere up to and following intense rainfall days

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
from cartopy.io import shapereader
import pandas as pd
import datetime
import matplotlib.patches as patches
import paraguayfloodspy.visualize as viz

Get some parameters and define others

In [ ]:
%matplotlib inline
savefigs = True

In [ ]:
days_back = [-2, -1, 0, 1]
%run ../config/PlotParameters.py
print(extent)
print(cmap)

In [ ]:
%run ../config/RioParaguay.mk
print(RPEAST, RPWEST, RPSOUTH, RPNORTH)

Get data: rainfall and streamfunction

In [ ]:
psi850 = xr.open_dataset("../data/derived/psi_850.nc")
psi200 = xr.open_dataset("../data/derived/psi_200.nc")
prcp = xr.open_dataset('../data/derived/precip.nc')

Get the area-averaged rainfall data in order to define rainy days as 95th percentile exceedances (can vary this parameter)

In [ ]:
prcp_rpy = pd.read_csv("../data/derived/rainfall_rpy.csv", index_col='time')['raw']
rainy_days = prcp_rpy.loc[prcp_rpy > np.nanpercentile(prcp_rpy, 95)].index
rainy_days = pd.to_datetime(rainy_days)

Define some required fields for plotting

In [ ]:
X,Y =  np.meshgrid(psi850.lon, psi850.lat)
Xp,Yp =  np.meshgrid(prcp.lon, prcp.lat)
ncols = len(days_back)

Specify how to draw a bounding box of the area over which rainfall is averaged to create the area-averaged rainfall!

In [ ]:
def rpy_rgn():
    rec = patches.Rectangle(
        (RPWEST-360, RPSOUTH),
        RPEAST - RPWEST,
        RPNORTH - RPSOUTH,
        color = 'black', label = 'Lower PY River Basin',
        fill = None, linewidth=2, transform = ccrs.PlateCarree()
        )
    return(rec)

Now make the plot

In [ ]:
fig, axes = viz.SetupAxes(ncol = ncols, nax = 3*ncols, proj = ccrs.Orthographic(-60, -10), figsize = [16, 8])
for i,d in enumerate(days_back):
    selector = lambda ds: ds.isel(time = np.in1d(ds.time, rainy_days + datetime.timedelta(days_back[i]))).mean(dim='time')
    #
    ax = axes[0, i]
    ax.set_title("t={} d".format(d))
    C0 = ax.contourf(X, Y, selector(psi200['anomaly']), transform = ccrs.PlateCarree(), 
                     cmap = cmap['psi_a'], extend="both", 
                     levels=np.linspace(-9e6, 9e6, 10))
    #
    ax = axes[1, i]
    C1 = ax.contourf(X, Y, selector(psi850['anomaly']), transform = ccrs.PlateCarree(), 
                     cmap = cmap['psi_a'], extend="both", 
                     levels=np.linspace(-3.5e6, 3.5e6, 8))
    #
    ax = axes[2, i]
    sub = selector(prcp['raw']).values
    sub = np.ma.masked_invalid(sub)
    C2 = ax.contourf(Xp, Yp, sub, transform = ccrs.PlateCarree(), 
                     cmap = cmap['rain'], extend = "max",
                     levels = np.linspace(0, 28, 8))
    ax.add_patch(rpy_rgn())
plt.tight_layout()
fig.subplots_adjust(right=0.94)
cax0 = fig.add_axes([0.97, 0.71, 0.01, 0.2])
cax1 = fig.add_axes([0.97, 0.39, 0.01, 0.2])
cax2 = fig.add_axes([0.97, 0.05, 0.01, 0.25])
cbar0 = fig.colorbar(C0, cax = cax0)
cbar0.formatter.set_powerlimits((7, 7))
cbar0.update_ticks() 
cbar0.set_label(r'$\psi_{200}$ Anomaly [$m^2$/s]', rotation=270)
cbar0.ax.get_yaxis().labelpad = 20
#
cbar1 = fig.colorbar(C1, cax = cax1)
cbar1.formatter.set_powerlimits((6, 6))
cbar1.update_ticks() 
cbar1.set_label(r'$\psi_{850}$ Anomaly [$m^2$/s]', rotation=270)
cbar1.ax.get_yaxis().labelpad = 20
# 
cbar2 = fig.colorbar(C2, cax=cax2)
cbar2.set_label('Precip. Anomaly [mm/d]', rotation=270)
cbar2.ax.get_yaxis().labelpad = 20
# 
viz.FormatAxes(axes[0,:], extent = extent['SH'])
viz.FormatAxes(axes[1,:], extent = extent['SH'])
viz.FormatAxes(axes[2,:], extent = extent['SAm'])
#
if savefigs:
    fig.savefig("../figs/LaggedRain.pdf", bbox_inches='tight')